<a href="https://colab.research.google.com/github/Dhairya18498/MIE1624_Assignment2/blob/main/parmar_1006859516_assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('drive/MyDrive/UofT/MIE1624_Intro_to_DS&Analytics/Assignment2')
!pwd

/content/drive/MyDrive/UofT/MIE1624_Intro_to_DS&Analytics/Assignment2


In [ ]:
cwd = os.getcwd()
files = os.listdir(cwd)
for f in files:
  print(f)

MIE1624-Assignment-2-Winter2023.pdf
clean_kaggle_data_2022.csv
parmar_1006859516_assignment2.ipynb


In [ ]:
import pandas as pd
# from IPython.display import HTML

survey_df = pd.read_csv('clean_kaggle_data_2022.csv', encoding = 'latin1')

# survey_df_html = survey_df.to_html()
# display(HTML(survey_df_html))

survey_df.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Duration (in seconds),Q2,Q3,Q4,Q5,Q6_1,Q6_2,Q6_3,Q6_4,Q6_5,...,Q44_5,Q44_6,Q44_7,Q44_8,Q44_9,Q44_10,Q44_11,Q44_12,Q29_Encoded,Q29_buckets
0,Duration (in seconds),What is your age (# years)?,What is your gender? - Selected Choice,In which country do you currently reside?,"Are you currently a student? (high school, uni...",On which platforms have you begun or completed...,On which platforms have you begun or completed...,On which platforms have you begun or completed...,On which platforms have you begun or completed...,On which platforms have you begun or completed...,...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...,Who/what are your favorite media sources that ...,NaN,NaN
1,851,55-59,Man,France,No,Coursera,NaN,Kaggle Learn Courses,NaN,NaN,...,"Course Forums (forums.fast.ai, Coursera forums...",NaN,NaN,"Blogs (Towards Data Science, Analytics Vidhya,...",NaN,NaN,NaN,NaN,2.0,"20,000-29,999"
2,501,30-34,Man,Germany,No,Coursera,edX,NaN,NaN,NaN,...,NaN,"YouTube (Kaggle YouTube, Cloud AI Adventures, ...",NaN,"Blogs (Towards Data Science, Analytics Vidhya,...",NaN,NaN,NaN,NaN,10.0,"100,000-124,999"
3,787,70+,Man,Australia,No,Coursera,NaN,Kaggle Learn Courses,NaN,NaN,...,NaN,NaN,"Podcasts (Chai Time Data Science, OEEReilly...",NaN,NaN,NaN,NaN,NaN,10.0,"100,000-124,999"
4,1132,40-44,Man,United States of America,No,Coursera,NaN,Kaggle Learn Courses,NaN,Fast.ai,...,NaN,NaN,"Podcasts (Chai Time Data Science, OEEReilly...","Blogs (Towards Data Science, Analytics Vidhya,...",NaN,NaN,NaN,NaN,13.0,"200,000-299,999"


In [ ]:
from IPython.display import HTML
survey_ques = pd.DataFrame(survey_df.loc[0])
survey_df.drop(0, inplace = True)
survey_ques.reset_index(inplace = True)
survey_ques_html = survey_ques.to_html()
display(HTML(survey_ques_html))

,index,0
0,Duration (in seconds),Duration (in seconds)
1,Q2,What is your age (# years)?
2,Q3,What is your gender? - Selected Choice
3,Q4,In which country do you currently reside?
4,Q5,"Are you currently a student? (high school, university, or graduate)"
5,Q6_1,On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Coursera
6,Q6_2,On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - edX
7,Q6_3,On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Kaggle Learn Courses
8,Q6_4,On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - DataCamp
9,Q6_5,On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Fast.ai


In [ ]:
survey_Q29 = survey_df[['Q29', 'Q29_Encoded', 'Q29_buckets']]
survey_Q29

,Q29,Q29_Encoded,Q29_buckets
1,"25,000-29,999",2.0,"20,000-29,999"
2,"100,000-124,999",10.0,"100,000-124,999"
3,"100,000-124,999",10.0,"100,000-124,999"
4,"200,000-249,999",13.0,"200,000-299,999"
5,"200,000-249,999",13.0,"200,000-299,999"
...,...,...,...
8132,"100,000-124,999",10.0,"100,000-124,999"
8133,"2,000-2,999",0.0,"0-9,999"
8134,$0-999,0.0,"0-9,999"
8135,"25,000-29,999",2.0,"20,000-29,999"


In [ ]:
import numpy as np
bucket_codes = list(np.unique(survey_Q29['Q29_Encoded']))
bucket_codes

[0.0,
 1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0]

In [ ]:
for code in bucket_codes:
  if isinstance(code, float):
    print(code, '--', survey_Q29.loc[survey_Q29.index[survey_Q29['Q29_Encoded'] == code][0], 'Q29_buckets'])

0.0 -- 0-9,999
1.0 -- 10,000-19,999
2.0 -- 20,000-29,999
3.0 -- 30,000-39,999
4.0 -- 40,000-49,999
5.0 -- 50,000-59,999
6.0 -- 60,000-69,999
7.0 -- 70,000-79,999
8.0 -- 80,000-89,999
9.0 -- 90,000-99,999
10.0 -- 100,000-124,999
11.0 -- 125,000-149,999
12.0 -- 150,000-199,999
13.0 -- 200,000-299,999
14.0 -- >300,000


In [ ]:
for column in list(survey_df.columns[survey_df.columns.str.contains('Q6')]):
  print(column, '--', np.unique(survey_df[column].astype(str)))

Q6_1 -- ['Coursera' 'nan']
Q6_2 -- ['edX' 'nan']
Q6_3 -- ['Kaggle Learn Courses' 'nan']
Q6_4 -- ['DataCamp' 'nan']
Q6_5 -- ['Fast.ai' 'nan']
Q6_6 -- ['Udacity' 'nan']
Q6_7 -- ['Udemy' 'nan']
Q6_8 -- ['LinkedIn Learning' 'nan']
Q6_9 -- ['Cloud-certification programs (direct from AWS, Azure, GCP, or similar)'
 'nan']
Q6_10 -- ['University Courses (resulting in a university degree)' 'nan']
Q6_11 -- ['None' 'nan']
Q6_12 -- ['Other' 'nan']


In [ ]:
MCQ_list = []
MCQ_cat_list = []
for q in range(2, 45):
  if len(list(survey_df.columns[survey_df.columns.str.contains('Q'+str(q)+'_')])) > 1:
    MCQ_list.append('Q'+str(q))
    for column in list(survey_df.columns[survey_df.columns.str.contains('Q'+str(q))]):
      if len(np.unique(survey_df[column].astype(str))) <= 2:
        # print(column, '--', np.unique(survey_df[column].astype(str)))
        MCQ_cat_list.append(column)
MCQ_list.remove('Q29')

Q6_1 -- ['Coursera' 'nan']
Q6_2 -- ['edX' 'nan']
Q6_3 -- ['Kaggle Learn Courses' 'nan']
Q6_4 -- ['DataCamp' 'nan']
Q6_5 -- ['Fast.ai' 'nan']
Q6_6 -- ['Udacity' 'nan']
Q6_7 -- ['Udemy' 'nan']
Q6_8 -- ['LinkedIn Learning' 'nan']
Q6_9 -- ['Cloud-certification programs (direct from AWS, Azure, GCP, or similar)'
 'nan']
Q6_10 -- ['University Courses (resulting in a university degree)' 'nan']
Q6_11 -- ['None' 'nan']
Q6_12 -- ['Other' 'nan']
Q7_1 -- ['University courses' 'nan']
Q7_2 -- ['Online courses (Coursera, EdX, etc)' 'nan']
Q7_3 -- ['Social media platforms (Reddit, Twitter, etc)' 'nan']
Q7_4 -- ['Video platforms (YouTube, Twitch, etc)' 'nan']
Q7_5 -- ['Kaggle (notebooks, competitions, etc)' 'nan']
Q7_6 -- ['None / I do not study data science' 'nan']
Q7_7 -- ['Other' 'nan']
Q10_1 -- ['Yes, the research made advances related to some novel machine learning method (theoretical research)'
 'nan']
Q10_2 -- ['Yes, the research made use of machine learning as a tool (applied research)'
 'nan']

In [ ]:
MCQ_cat_list

['Q6_1',
 'Q6_2',
 'Q6_3',
 'Q6_4',
 'Q6_5',
 'Q6_6',
 'Q6_7',
 'Q6_8',
 'Q6_9',
 'Q6_10',
 'Q6_11',
 'Q6_12',
 'Q7_1',
 'Q7_2',
 'Q7_3',
 'Q7_4',
 'Q7_5',
 'Q7_6',
 'Q7_7',
 'Q10_1',
 'Q10_2',
 'Q10_3',
 'Q12_1',
 'Q12_2',
 'Q12_3',
 'Q12_4',
 'Q12_5',
 'Q12_6',
 'Q12_7',
 'Q12_8',
 'Q12_9',
 'Q12_10',
 'Q12_11',
 'Q12_12',
 'Q12_13',
 'Q12_14',
 'Q12_15',
 'Q13_1',
 'Q13_2',
 'Q13_3',
 'Q13_4',
 'Q13_5',
 'Q13_6',
 'Q13_7',
 'Q13_8',
 'Q13_9',
 'Q13_10',
 'Q13_11',
 'Q13_12',
 'Q13_13',
 'Q13_14',
 'Q14_1',
 'Q14_2',
 'Q14_3',
 'Q14_4',
 'Q14_5',
 'Q14_6',
 'Q14_7',
 'Q14_8',
 'Q14_9',
 'Q14_10',
 'Q14_11',
 'Q14_12',
 'Q14_13',
 'Q14_14',
 'Q14_15',
 'Q14_16',
 'Q15_1',
 'Q15_2',
 'Q15_3',
 'Q15_4',
 'Q15_5',
 'Q15_6',
 'Q15_7',
 'Q15_8',
 'Q15_9',
 'Q15_10',
 'Q15_11',
 'Q15_12',
 'Q15_13',
 'Q15_14',
 'Q15_15',
 'Q17_1',
 'Q17_2',
 'Q17_3',
 'Q17_4',
 'Q17_5',
 'Q17_6',
 'Q17_7',
 'Q17_8',
 'Q17_9',
 'Q17_10',
 'Q17_11',
 'Q17_12',
 'Q17_13',
 'Q17_14',
 'Q17_15',
 'Q18_1',
 'Q18_